In [1]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
import csv
import os
from copy import deepcopy

from datetime import datetime

In [2]:
root = "/data/ephemeral/home/datasets/UCFCrime/normal/"


header = [
    "Filename",
    "Frame",
    "ID",
    "X",
    "Y",
    "Width",
    "Height",
    "Keypoint_0",
    "Keypoint_1",
    "Keypoint_2",
    "Keypoint_3",
    "Keypoint_4",
    "Keypoint_5",
    "Keypoint_6",
    "Keypoint_7",
    "Keypoint_8",
    "Keypoint_9",
    "Keypoint_10",
    "Keypoint_11",
    "Keypoint_12",
    "Keypoint_13",
    "Keypoint_14",
    "Keypoint_15",
    "Keypoint_16",
    "Keypoint_17",
    "Keypoint_18",
    "Keypoint_19",
    "Keypoint_20",
    "Keypoint_21",
    "Keypoint_22",
    "Keypoint_23",
    "Keypoint_24",
    "Keypoint_25",
    "Keypoint_26",
    "Keypoint_27",
    "Keypoint_28",
    "Keypoint_29",
    "Keypoint_30",
    "Keypoint_31",
    "Keypoint_32",
    "Keypoint_33",
]

In [3]:
# Load the YOLOv8 model
model = YOLO("yolov8n-pose.pt")

# Define the standard frame size (change these values as needed)
standard_width = 320
standard_height = 240

In [4]:
folder_list = os.listdir(root)
folder_list.sort()
print(f"==>> folder_list: {folder_list}")

==>> folder_list: ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']


In [5]:
for folder_name in folder_list:
    time_start = datetime.now()

    print(f"{folder_name} feature extracting starts")

    with open(f"{folder_name}.csv", "w") as c_file:
        writer = csv.writer(c_file, delimiter=",")

        writer.writerow(header)

    folder_path = root + folder_name + "/"

    file_list = os.listdir(root+folder_name)
    file_list.sort()
    print(f"==>> file_list: {file_list}")

    id_count = 0

    for file_name in file_list:
        path = folder_path + file_name

        cap = cv2.VideoCapture(path)

        # Loop through the video frames
        frame_count = 0

        # Store the track history
        track_history = defaultdict(lambda: [])

        while cap.isOpened():
            # Read a frame from the video
            success, frame = cap.read()

            frame_count += 1  # Increment frame count

            if success:
                frame = cv2.resize(frame, (standard_width, standard_height))

                # Run YOLOv8 tracking on the frame, persisting tracks between frames
                results = model.track(frame, persist=True, verbose=False)

                if results[0].boxes is not None:  # Check if there are results and boxes
                    # Get the boxes
                    # boxes = results[0].boxes.xywh.cpu()

                    if results[0].boxes.id is not None:
                        # If 'int' attribute exists (there are detections), get the track IDs
                        track_ids = results[0].boxes.id.int().cpu().tolist()

                        for i, box in zip(range(0, len(track_ids)), results[0].boxes.xywhn.cpu()):
                            keypoints = results[0].keypoints.xyn[i].cpu().numpy().flatten().tolist()
                            box_list = box.numpy().flatten().tolist()
                            if type(box_list) == "float" or type(keypoints) == "float":
                                print(f"==>> box_list: {box_list}")
                                print(f"==>> keypoints: {keypoints}")
                            box_and_keypoints = box_list + keypoints
                            track_history[track_ids[i]].append([[frame_count], deepcopy(box_and_keypoints)])
            else:
                # Break the loop if the end of the video is reached
                break

        with open(f"{folder_name}.csv", "a") as c_file:
            writer = csv.writer(c_file, delimiter=",")
            for key in track_history.keys():
                for f_count, b_and_k in track_history[key]:
                    row = [file_name] + f_count + [id_count + key] + b_and_k

                    writer.writerow(row)

        id_count = id_count + len(track_history.keys())

        cap.release()

    time_end = datetime.now()
    total_time = time_end - time_start
    total_time = str(total_time).split(".")[0]

    print(f"{folder_name} feature extracting ended. Elapsed time: {total_time}")

# cv2.destroyAllWindows()

Abuse feature extracting starts
==>> file_list: ['Abuse001_x264.mp4', 'Abuse002_x264.mp4', 'Abuse003_x264.mp4', 'Abuse004_x264.mp4', 'Abuse005_x264.mp4', 'Abuse006_x264.mp4', 'Abuse007_x264.mp4', 'Abuse008_x264.mp4', 'Abuse009_x264.mp4', 'Abuse010_x264.mp4', 'Abuse011_x264.mp4', 'Abuse012_x264.mp4', 'Abuse013_x264.mp4', 'Abuse014_x264.mp4', 'Abuse015_x264.mp4', 'Abuse016_x264.mp4', 'Abuse017_x264.mp4', 'Abuse018_x264.mp4', 'Abuse019_x264.mp4', 'Abuse020_x264.mp4', 'Abuse021_x264.mp4', 'Abuse022_x264.mp4', 'Abuse023_x264.mp4', 'Abuse024_x264.mp4', 'Abuse025_x264.mp4', 'Abuse026_x264.mp4', 'Abuse027_x264.mp4', 'Abuse028_x264.mp4', 'Abuse029_x264.mp4', 'Abuse030_x264.mp4', 'Abuse031_x264.mp4', 'Abuse032_x264.mp4', 'Abuse033_x264.mp4', 'Abuse034_x264.mp4', 'Abuse035_x264.mp4', 'Abuse036_x264.mp4', 'Abuse037_x264.mp4', 'Abuse038_x264.mp4', 'Abuse039_x264.mp4', 'Abuse040_x264.mp4', 'Abuse041_x264.mp4', 'Abuse042_x264.mp4', 'Abuse043_x264.mp4', 'Abuse044_x264.mp4', 'Abuse045_x264.mp4', 'Abuse